# Comparison Between TreeValue and Tianshou Batch

In this section, we will take a look at the feature and performance of the [Tianshou Batch](https://github.com/thu-ml/tianshou) library, which is developed by Tsinghua Machine Learning Group.

Before starting the comparison, let us define some thing.

In [1]:
import torch 

_TREE_DATA_1 = {'a': 1, 'b': 2, 'x': {'c': 3, 'd': 4}}
_TREE_DATA_2 = {
    'a': torch.randn(2, 3), 
    'x': {
        'c': torch.randn(3, 4)
    },
}
_TREE_DATA_3 = {
    'obs': torch.randn(4, 84, 84),
    'action': torch.randint(0, 6, size=(1,)),
    'reward': torch.rand(1),
}

/opt/hostedtoolcache/Python/3.8.15/x64/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Read and Write Operation

Reading and writing are the two most common operations in the tree data structure based on the data model (TreeValue and Tianshou Batch both belong to this type), so this section will compare the reading and writing performance of these two libraries.

### TreeValue's Get and Set

In [2]:
from treevalue import FastTreeValue

t = FastTreeValue(_TREE_DATA_2)

In [3]:
t

<FastTreeValue 0x7f8288a67640>
├── 'a' --> tensor([[ 1.7529, -0.5754, -0.0720],
│                   [ 0.0896, -0.6951, -0.8790]])
└── 'x' --> <FastTreeValue 0x7f8288a676a0>
    └── 'c' --> tensor([[-0.2025,  0.0488, -0.1097, -1.0921],
                        [-1.8141,  1.0614, -0.1761,  0.4995],
                        [ 0.1470, -0.1327, -0.9774, -0.9220]])

In [4]:
t.a

tensor([[ 1.7529, -0.5754, -0.0720],
        [ 0.0896, -0.6951, -0.8790]])

In [5]:
%timeit t.a

67 ns ± 0.0802 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [6]:
new_value = torch.randn(2, 3)
t.a = new_value

t

<FastTreeValue 0x7f8288a67640>
├── 'a' --> tensor([[-0.1561, -0.1399,  2.6812],
│                   [ 0.7328, -2.0681,  0.6336]])
└── 'x' --> <FastTreeValue 0x7f8288a676a0>
    └── 'c' --> tensor([[-0.2025,  0.0488, -0.1097, -1.0921],
                        [-1.8141,  1.0614, -0.1761,  0.4995],
                        [ 0.1470, -0.1327, -0.9774, -0.9220]])

In [7]:
%timeit t.a = new_value

65.1 ns ± 0.0474 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


### Tianshou Batch's Get and Set

In [8]:
from tianshou.data import Batch

b = Batch(**_TREE_DATA_2)

In [9]:
b

Batch(
    a: tensor([[ 1.7529, -0.5754, -0.0720],
               [ 0.0896, -0.6951, -0.8790]]),
    x: Batch(
           c: tensor([[-0.2025,  0.0488, -0.1097, -1.0921],
                      [-1.8141,  1.0614, -0.1761,  0.4995],
                      [ 0.1470, -0.1327, -0.9774, -0.9220]]),
       ),
)

In [10]:
b.a

tensor([[ 1.7529, -0.5754, -0.0720],
        [ 0.0896, -0.6951, -0.8790]])

In [11]:
%timeit b.a

59.6 ns ± 0.0535 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [12]:
new_value = torch.randn(2, 3)
b.a = new_value

b

Batch(
    a: tensor([[-0.4126,  0.2466,  0.5372],
               [-0.7537, -0.5100, -0.0215]]),
    x: Batch(
           c: tensor([[-0.2025,  0.0488, -0.1097, -1.0921],
                      [-1.8141,  1.0614, -0.1761,  0.4995],
                      [ 0.1470, -0.1327, -0.9774, -0.9220]]),
       ),
)

In [13]:
%timeit b.a = new_value

493 ns ± 0.167 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


## Initialization

### TreeValue's Initialization

In [14]:
%timeit FastTreeValue(_TREE_DATA_1)

820 ns ± 0.0737 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


### Tianshou Batch's Initialization

In [15]:
%timeit Batch(**_TREE_DATA_1)

10.9 µs ± 20.4 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


## Deep Copy Operation

In [16]:
import copy

### Deep Copy of TreeValue

In [17]:
t3 = FastTreeValue(_TREE_DATA_3)
%timeit copy.deepcopy(t3)

145 µs ± 1.46 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Deep Copy of Tianshou Batch

In [18]:
b3 = Batch(**_TREE_DATA_3)
%timeit copy.deepcopy(b3)

145 µs ± 930 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


## Stack, Concat and Split Operation

### Performance of TreeValue

In [19]:
trees = [FastTreeValue(_TREE_DATA_2) for _ in range(8)]

In [20]:
t_stack = FastTreeValue.func(subside=True)(torch.stack)

t_stack(trees)

<FastTreeValue 0x7f81dff80e20>
├── 'a' --> tensor([[[ 1.7529, -0.5754, -0.0720],
│                    [ 0.0896, -0.6951, -0.8790]],
│           
│                   [[ 1.7529, -0.5754, -0.0720],
│                    [ 0.0896, -0.6951, -0.8790]],
│           
│                   [[ 1.7529, -0.5754, -0.0720],
│                    [ 0.0896, -0.6951, -0.8790]],
│           
│                   [[ 1.7529, -0.5754, -0.0720],
│                    [ 0.0896, -0.6951, -0.8790]],
│           
│                   [[ 1.7529, -0.5754, -0.0720],
│                    [ 0.0896, -0.6951, -0.8790]],
│           
│                   [[ 1.7529, -0.5754, -0.0720],
│                    [ 0.0896, -0.6951, -0.8790]],
│           
│                   [[ 1.7529, -0.5754, -0.0720],
│                    [ 0.0896, -0.6951, -0.8790]],
│           
│                   [[ 1.7529, -0.5754, -0.0720],
│                    [ 0.0896, -0.6951, -0.8790]]])
└── 'x' --> <FastTreeValue 0x7f8288a2e700>
    └── 'c' --> tensor([[[

In [21]:
%timeit t_stack(trees)

32.4 µs ± 117 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [22]:
t_cat = FastTreeValue.func(subside=True)(torch.cat)

t_cat(trees)

<FastTreeValue 0x7f8288ad3d30>
├── 'a' --> tensor([[ 1.7529, -0.5754, -0.0720],
│                   [ 0.0896, -0.6951, -0.8790],
│                   [ 1.7529, -0.5754, -0.0720],
│                   [ 0.0896, -0.6951, -0.8790],
│                   [ 1.7529, -0.5754, -0.0720],
│                   [ 0.0896, -0.6951, -0.8790],
│                   [ 1.7529, -0.5754, -0.0720],
│                   [ 0.0896, -0.6951, -0.8790],
│                   [ 1.7529, -0.5754, -0.0720],
│                   [ 0.0896, -0.6951, -0.8790],
│                   [ 1.7529, -0.5754, -0.0720],
│                   [ 0.0896, -0.6951, -0.8790],
│                   [ 1.7529, -0.5754, -0.0720],
│                   [ 0.0896, -0.6951, -0.8790],
│                   [ 1.7529, -0.5754, -0.0720],
│                   [ 0.0896, -0.6951, -0.8790]])
└── 'x' --> <FastTreeValue 0x7f8288a6c4c0>
    └── 'c' --> tensor([[-0.2025,  0.0488, -0.1097, -1.0921],
                        [-1.8141,  1.0614, -0.1761,  0.4995],
                 

In [23]:
%timeit t_cat(trees)

30.6 µs ± 53.2 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [24]:
t_split = FastTreeValue.func(rise=True)(torch.split)
tree = FastTreeValue({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1),
})

%timeit t_split(tree, 1)

55.9 µs ± 187 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Performance of Tianshou Batch

In [25]:
batches = [Batch(**_TREE_DATA_2) for _ in range(8)]

Batch.stack(batches)

Batch(
    a: tensor([[[ 1.7529, -0.5754, -0.0720],
                [ 0.0896, -0.6951, -0.8790]],
       
               [[ 1.7529, -0.5754, -0.0720],
                [ 0.0896, -0.6951, -0.8790]],
       
               [[ 1.7529, -0.5754, -0.0720],
                [ 0.0896, -0.6951, -0.8790]],
       
               [[ 1.7529, -0.5754, -0.0720],
                [ 0.0896, -0.6951, -0.8790]],
       
               [[ 1.7529, -0.5754, -0.0720],
                [ 0.0896, -0.6951, -0.8790]],
       
               [[ 1.7529, -0.5754, -0.0720],
                [ 0.0896, -0.6951, -0.8790]],
       
               [[ 1.7529, -0.5754, -0.0720],
                [ 0.0896, -0.6951, -0.8790]],
       
               [[ 1.7529, -0.5754, -0.0720],
                [ 0.0896, -0.6951, -0.8790]]]),
    x: Batch(
           c: tensor([[[-0.2025,  0.0488, -0.1097, -1.0921],
                       [-1.8141,  1.0614, -0.1761,  0.4995],
                       [ 0.1470, -0.1327, -0.9774, -0.9220]],
         

In [26]:
%timeit Batch.stack(batches)

79.3 µs ± 258 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [27]:
Batch.cat(batches)

Batch(
    a: tensor([[ 1.7529, -0.5754, -0.0720],
               [ 0.0896, -0.6951, -0.8790],
               [ 1.7529, -0.5754, -0.0720],
               [ 0.0896, -0.6951, -0.8790],
               [ 1.7529, -0.5754, -0.0720],
               [ 0.0896, -0.6951, -0.8790],
               [ 1.7529, -0.5754, -0.0720],
               [ 0.0896, -0.6951, -0.8790],
               [ 1.7529, -0.5754, -0.0720],
               [ 0.0896, -0.6951, -0.8790],
               [ 1.7529, -0.5754, -0.0720],
               [ 0.0896, -0.6951, -0.8790],
               [ 1.7529, -0.5754, -0.0720],
               [ 0.0896, -0.6951, -0.8790],
               [ 1.7529, -0.5754, -0.0720],
               [ 0.0896, -0.6951, -0.8790]]),
    x: Batch(
           c: tensor([[-0.2025,  0.0488, -0.1097, -1.0921],
                      [-1.8141,  1.0614, -0.1761,  0.4995],
                      [ 0.1470, -0.1327, -0.9774, -0.9220],
                      [-0.2025,  0.0488, -0.1097, -1.0921],
                      [-1.8141,  

In [28]:
%timeit Batch.cat(batches)

146 µs ± 182 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [29]:
batch = Batch({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1)}
)

%timeit list(Batch.split(batch, 1, shuffle=False, merge_last=True))

328 µs ± 1.65 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
